## 1. Connect to Azure OpenAI LLM
Initialize and connect to the Azure OpenAI service using credentials from environment variables to enable chat completions.

In [16]:
import os
load_dotenv()

True

In [ ]:
#connect to llm
from openai import AzureOpenAI
from dotenv import load_dotenv


client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_BASE_URL"),
    api_version="2024-08-01-preview"
)

response = client.chat.completions.create(   
  model="gpt-4o",#  Replace with your actual deployment name from Azure Portal
  messages=[
    {"role": "user", "content": "This is a test."}
  ]
)

print(response.model_dump_json(indent=2))

## 2. Install Azure Search Documents Library
Install the required `azure-search-documents` package to interact with Azure AI Search service.
- pip install azure-search-documents

## 3. Connect to Azure AI Search Service
Initialize the SearchClient with endpoint, index name, and API key from environment variables to enable queries against the Azure Search index.

In [22]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

## 4. Search and Display Results
Execute a search query against the Azure Search index and display all results with their fields and values.

In [ ]:
results = search_client.search(search_text="ROBERT AUDI")

print("Checking results...")
result_list = list(results)
print(f"Number of results: {len(result_list)}")

if result_list:
    print("\nFirst result fields:")
    print(result_list[0])
    
    for result in result_list:
        print("\n---")
        for key, value in result.items():
            print(f"{key}: {value}")
else:
    print("No results found")